# Deve ser executado com o acelerador TPU
Edit-> Notebook Settings -> selecionar TPU

In [0]:
import tensorflow as tf
import numpy as np
from copy import deepcopy


Necessário para a importacao do model

In [6]:
from google.colab import drive
drive.mount('content/drive')


ValueError: ignored

In [8]:
 drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


#

In [0]:
drive.umount("/content/drive", force_remount=True)

Mounted at /content/drive


In [0]:
from model_low_sparse import Model


# Metodo utilizado para a criacao dos dados binarios randomicos

In [0]:
def buildRandomBoolDataSet(Nsamples=2000,Dinput=1000,seed=0):
    np.random.seed(seed)
    features, labels = (np.random.randint(2,size=(Nsamples,Dinput)), np.eye(Nsamples))
    features = (np.float32(features)*2-1)/np.sqrt(Dinput)
    labels = np.float32(labels)
    dataset = tf.data.Dataset.from_tensor_slices((features,labels))
    return dataset
  


In [19]:
sess = tf.InteractiveSession()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [23]:
    
#Nsamples = 1000
#list_samples = np.linspace(1,1000,10,dtype=int)
#input = 100
#learning_rate = 0.1
#model_seed = 0
#graph_samples = np.zeros(len(list_samples))
#nsteps = 1000
#sparseness = 0.4




input_dim = 100
#num_samples = 100
#sparseness = 0.9
learning_rate = 0.1
model_seed = 0
data_seed = 0
nsteps = 1000
max_count = 100


sparse = np.linspace(0.0,0.9,10)

num_samplesV = np.power(10,np.arange(0.2,4.1,0.2))
data_seedV = np.arange(20)

acc = np.ones((len(sparse),len(num_samplesV),len(data_seedV))) * np.nan

for zz,sparseness in enumerate(sparse):
    for ii,num_samples in enumerate(num_samplesV):
        for jj,data_seed in enumerate(data_seedV):
                
                myDataSet = buildRandomBoolDataSet(num_samples.astype(np.int64),input_dim, data_seed).batch(num_samples)                
                myIter = myDataSet.make_initializable_iterator()
                myFeatures, myLabels = myIter.get_next()               
                x = myFeatures
                y_ = myLabels              
                #model = Model(x, y_, spars, 0.1) # simple 2-layer network
                model = Model(x, y_, sparseness, learning_rate, seed=data_seed) # simple 2-layer network
                model.set_vanilla_loss()              
                # initialize variables
                sess.run(tf.global_variables_initializer())    
                sess.run(myIter.initializer)                
                count  = 1            
                for ss in np.arange(1,nsteps+1):
                    sess.run(myIter.initializer)
                    model.train_step.run()
                    sess.run(myIter.initializer)
                    #acc[ss] = model.accuracy.eval()
                    
                    acc[zz,ii,jj] = model.accuracy.eval()

AttributeError: ignored

Gerando o grafico esparsialidade versus num Sample

In [0]:
ax = plt.subplot(111)

for zz,sparseness in enumerate(sparse):  
    plt.semilogx(num_samplesV,np.median(acc[zz,:,:],axis=1), label="spars=%1.1f"%(sparseness,))

plt.xlabel('NSamples ')
plt.ylabel('Accuracy')
plt.grid(True)
leg = plt.legend(bbox_to_anchor=(1.1, 1.05))
leg.get_frame().set_alpha(0.5)
plt.show()

[0.002      0.048      0.27700001 ... 1.         1.         1.        ]
